In [1]:
from sklearn.svm import SVC
import pennylane.numpy as np
from model_parameters import ModelParameters
import time
from helper.scores import Metrics
from helper.dataset import sub_select_dataset
from helper.svm import run_svm_sequence

# Training SVM

In [2]:

metrics = Metrics()
model_parameters = ModelParameters()
x_train, x_test, y_train, y_test = model_parameters.get_htru_2()

model = SVC(kernel='linear', C=1.0) 

run_svm_sequence(model_parameters, model, x_train, y_train, x_test, y_test, metrics)

metrics.display()

Pour 6 runs:
Accuracy: 0.972 ± 0.012
Balanced Accuracy: 0.926 ± 0.041
Recall: 0.872 ± 0.076
Specificity: 0.980 ± 0.008
Precision: 0.797 ± 0.083
NPV: 0.989 ± 0.006
G-Mean: 0.924 ± 0.044
Informedness: 0.853 ± 0.082
Training Duration: 0.001 ± 0.000
Testing Duration: 0.000 ± 0.000
